In [1]:
import numpy as np
import xarray as xr
import geopandas as gpd
import pandas as pd
from matplotlib import pyplot as plt
from osgeo import gdal, ogr, gdal_array
import dask
import datacube 
from datacube.helpers import ga_pq_fuser
from datacube.storage import masking
from datacube.utils import geometry
from multiprocessing import Pool, cpu_count
#import custom functions
import sys
sys.path.append('src')
import DEAPlotting, SpatialTools, BandIndices, DEADataHandling

from query_from_shp import query_from_shp

import fiona
import rasterio.features

import warnings
warnings.filterwarnings('ignore')

### user inputs

In [8]:
results = "results/"
AOI = 'WaggaWagga'
year= '2018'

results = results + AOI + "_" + year + "/"

time_period = ('1990-01-01', '2019-01-01')

shp_path = results + 'WaggaWagga_2018_imageSeg_cultivated_polygons_withID.shp'

### Paddock Drill

In [9]:
dc = datacube.Datacube(app='fc_fun')

with fiona.open(shp_path) as input:
    crs = geometry.CRS(input.crs_wkt)
    
def paddockDrill(feat, crs):
    first_geom = feat['geometry']
    poly_id = feat['properties']['id']
    progress = round((poly_id/8412) * 100, 4)
    print("\r", "working on polygon: " + str(poly_id) + ", " + str(progress) + "%" + " complete. ", end = '')
    geom = geometry.Geometry(first_geom, crs=crs)

    query = {'geopolygon': geom,
             'time': time_period}

    data = DEADataHandling.load_clearlandsat(dc, query,product='fc', ls7_slc_off =True, masked_prop=0.90)

    mask = rasterio.features.geometry_mask([geom.to_crs(data.geobox.crs)for geoms in [geom]],
                                       out_shape=data.geobox.shape,
                                       transform=data.geobox.affine,
                                       all_touched=False,
                                       invert=False)

    mask_xr = xr.DataArray(mask, dims = ('y','x'))
    fc = data.where(mask_xr==False)

    PV_mean = fc.PV.mean(dim=['x', 'y']).values
    PV_std = fc.PV.std(dim=['x', 'y']).values
    NPV_mean = fc.NPV.mean(dim=['x', 'y']).values
    BS_mean = fc.BS.mean(dim=['x', 'y']).values        
    time = fc.time.values

    #export csv
    statistics_df = pd.DataFrame({'dateTime': time, 
                                  'Mean PV (%)': PV_mean, 
                                  'Std. Dev. PV (%)':PV_std,
                                  'Mean NPV (%)': NPV_mean,
                                  'Mean BS (%)': BS_mean
                                 })
    statistics_df.to_csv(results + "paddock_zonal/" + str(poly_id) + ".csv")

p = Pool(max(cpu_count()//2, 1)) #this will use half the processors on the computer
for feat in fiona.open(shp_path):
    p.apply_async(paddockDrill, [feat, crs])  


 working on polygon: 2, 0.0238% complete. Loading ls5 pixel quality
 working on polygon: 3, 0.0357% complete. Loading ls5 pixel quality
 working on polygon: 1, 0.0119% complete. Loading ls5 pixel quality
 working on polygon: 4, 0.0476% complete. Loading ls5 pixel quality
    Skipping ls5; no valid data for query



finished
